In [2]:
import numpy as np
import keras
import tensorflow as tf
from keras import layers
from keras import models
from keras.datasets import california_housing

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary
from torch.optim.lr_scheduler import StepLR

from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

import sys
import math

import numpy as np
import math

import numpy as np
import keras
import tensorflow as tf
from keras import layers
from keras import models
from keras.datasets import mnist

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchinfo import summary
from torch.optim.lr_scheduler import StepLR

import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

import numpy as np
import pandas as pd
from scipy.special import logit
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import LeaveOneOut, cross_val_predict, StratifiedKFold
from scipy.stats import norm

In [3]:
def calculate_metrics(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    return accuracy, f1

In [4]:
class CustomDataLoader:
    def __init__(self, features, labels, batch_size=1, validation_size=0.0, shuffle=False):

        if validation_size > 0:
            train_data, val_data, train_labels, val_labels = train_test_split(features, labels, test_size=validation_size, stratify=labels, random_state=42)
            train_data_tensor = torch.tensor(train_data).float().to(device)
            train_labels_tensor = torch.tensor(train_labels).long().to(device)
            val_data_tensor = torch.tensor(val_data).float().to(device)
            val_labels_tensor = torch.tensor(val_labels).long().to(device)
    
            train_dataset = TensorDataset(train_data_tensor, train_labels_tensor)
            val_dataset = TensorDataset(val_data_tensor, val_labels_tensor)

            self.train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
            self.val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        else:
            features_tensor = torch.tensor(features).float().to(device)
            labels_tensor = torch.tensor(labels).long().to(device)

            dataset = TensorDataset(features_tensor, labels_tensor)

            self.train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
            self.val_loader = None

    def get_train_loader(self):
        return self.train_loader
    
    def get_val_loader(self):
        return self.val_loader

In [176]:
def evaluate_model(model, custom_train_loader, criterion, optimizer):
    num_epochs = 1000
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in custom_train_loader.get_train_loader():        
            optimizer.zero_grad()
            outputs = model(inputs.view(-1, 12 * 1))
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}, Loss: {running_loss / len(custom_train_loader.get_train_loader())}')

            model.eval()
            running_val_loss = 0.0
            with torch.no_grad():
                for inputs, labels in custom_train_loader.get_val_loader():
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs.view(-1, 12 * 1))
                    val_loss = criterion(outputs, labels)
                    running_val_loss += val_loss.item()

                avg_val_loss = running_val_loss / len(custom_train_loader.get_val_loader())
                print(f'Validation Loss: {avg_val_loss}')
            
            val_accuracy, val_f1 = calculate_metrics(model, custom_train_loader.get_val_loader())
            print(f'Val Accuracy: {val_accuracy:.4f}, Val F1-score: {val_f1:.4f}')
            print()

In [177]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [178]:
data = pd.read_csv('/kaggle/input/brain-tumor/Brain Tumor.csv')
data = data.dropna()

In [189]:
X = data.drop(
    [
        "Image",
        "Class",
    ],
    axis=1,
)
y = data["Class"]

In [192]:
X.columns
column_count = X.shape[1]
column_count

12

In [191]:
X['log_mean'] = np.log(X['Mean'])
X['log_variance'] = np.log(X['Variance'])
X['log_std_dev'] = np.log(X['Standard Deviation'])
X['log_entropy'] = np.log(X['Entropy'])  # possibly not need to take log
X['log_skewness'] = np.log(X['Skewness']) # possibly not need to take log
X['log_kurtosis'] = np.log(X['Kurtosis'])
X['log_contrast'] = np.log(X['Contrast'])
X['log_energy'] = np.log(X['Energy'] / (1 - X['Energy'])) # may take this out
X['log_asm'] = np.log(X['ASM'] / (1 - X['ASM'])) # may take this out
X['log_homogeneity'] = np.log(X['Homogeneity'] / (1 - X['Homogeneity'])) # may take this out
X['log_dissimilarity'] = np.log(X['Dissimilarity'])
X['log_correlation'] = np.log(X['Correlation'] / (1 - X['Correlation']))

new_columns = [
    'log_mean', 'log_variance', 'log_std_dev', 'log_entropy', 'log_skewness', 
    'log_kurtosis', 'log_contrast', 'log_energy', 'log_asm', 'log_homogeneity', 
    'log_dissimilarity', 'log_correlation'
]

X = X.loc[:, new_columns]

# negative_columns = X * -1
# negative_columns.columns = [f"negative_{col}" for col in new_columns]

# X = pd.concat([X, negative_columns], axis=1)

In [193]:
x_scaler = StandardScaler()
x_scaled = x_scaler.fit_transform(X)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [194]:
custom_train_loader = CustomDataLoader(x_scaled, y_encoded, batch_size=16, validation_size=0.2, shuffle=True)

In [201]:
import torch
import torch.nn as nn
from numpy import pi

class CustomLayer1(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CustomLayer1, self).__init__()
        self.sigmoid = nn.Sigmoid()
        self.hard_sigmoid = nn.Hardsigmoid()
        self.linear_sigmoid = nn.Linear(input_dim, output_dim)
        self.relu = nn.ReLU()
        self.softplus = nn.Softplus()
        self.tanh = nn.Tanh()

        nn.init.zeros_(self.linear_sigmoid.weight)
        nn.init.zeros_(self.linear_sigmoid.bias)

    def forward(self, x, prev_x):
#         sigmoid_gate = self.sigmoid(2 * self.linear_sigmoid(prev_x))
#         return x * sigmoid_gate * 2
#         cdf_gate = 1 + torch.erf(self.linear_sigmoid(prev_x) / torch.sqrt(torch.tensor(2.0)))
        err_gate = 1 + torch.erf(self.linear_sigmoid(prev_x) * np.sqrt(pi) / 2)
        return x * err_gate

In [210]:
class DenseMLP(nn.Module):
    def __init__(self, input_size, num_layers, output_size):
        super(DenseMLP, self).__init__()        
        self.num_layers = num_layers
        self.layers = nn.ModuleList()
        self.custom_layers = nn.ModuleList()
   
        layer_size = input_size
        for i in range(1, num_layers):
            self.layers.append(nn.Linear(layer_size, layer_size))
            self.custom_layers.append(CustomLayer1(layer_size, layer_size))
            layer_size *= 2
    
        self.layers.append(nn.Linear(layer_size, output_size))       
        self.custom_layers.append(CustomLayer1(layer_size, output_size))
        self._initialize_weights()

    def forward(self, x):
        outputs = [x]
        
        for layer, custom_layer in zip(self.layers, self.custom_layers):
            concatenated_outputs = torch.cat(outputs, dim=1)
            out = layer(concatenated_outputs)
            out = custom_layer(out, concatenated_outputs)
            outputs.append(out)

        return outputs[-1]
    
    def _initialize_weights(self):
        for i, layer in enumerate(self.layers):
            if isinstance(layer, nn.Linear):
                if i == len(self.layers) - 1:
                    layer.weight.data.fill_(0)
                else:
                    in_features = layer.weight.size(1)
                    eye_matrix = torch.eye(in_features)
                    layer.weight.data = eye_matrix
                if layer.bias is not None:
                    nn.init.normal_(layer.bias, mean=0.0, std=1.0)
                    layer.bias.data.zero_()

model = DenseMLP(12, 2, 2).to(device)
criterion = nn.CrossEntropyLoss()
summary(model, input_size=(1,12))

Layer (type:depth-idx)                   Output Shape              Param #
DenseMLP                                 [1, 2]                    --
├─ModuleList: 1-3                        --                        (recursive)
│    └─Linear: 2-1                       [1, 12]                   156
├─ModuleList: 1-4                        --                        (recursive)
│    └─CustomLayer1: 2-2                 [1, 12]                   --
│    │    └─Linear: 3-1                  [1, 12]                   156
├─ModuleList: 1-3                        --                        (recursive)
│    └─Linear: 2-3                       [1, 2]                    50
├─ModuleList: 1-4                        --                        (recursive)
│    └─CustomLayer1: 2-4                 [1, 2]                    --
│    │    └─Linear: 3-2                  [1, 2]                    50
Total params: 412
Trainable params: 412
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forwa

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
evaluate_model(model, custom_train_loader, criterion, optimizer)

Epoch 10, Loss: 0.03374789767039052
Validation Loss: 0.018516283520644567
Val Accuracy: 0.9973, Val F1-score: 0.9973

Epoch 20, Loss: 0.025564137760444347
Validation Loss: 0.017076328937856516
Val Accuracy: 0.9960, Val F1-score: 0.9960

Epoch 30, Loss: 0.017167417661641225
Validation Loss: 0.01940715803722422
Val Accuracy: 0.9960, Val F1-score: 0.9960



In [175]:
for name, param in model.named_parameters():
    print(f"{name}: {param.data}")

layers.0.weight: tensor([[-6.8681, -0.2523, -0.2523, -2.5118, -0.1016, -5.4633,  5.3171,  0.5473,
         -1.4294,  2.5647, -0.6739,  2.9581,  6.8681,  0.2523,  0.2523,  2.5118,
          0.1016,  5.4633, -5.3171, -0.5473,  1.4294, -2.5647,  0.6739, -2.9581],
        [ 2.4487,  1.1285,  1.1285, -1.4642,  1.8403,  5.7803,  1.2572, -1.7371,
         -1.5784,  0.8376, -2.1668, -0.0709, -2.4487, -1.1285, -1.1285,  1.4642,
         -1.8403, -5.7803, -1.2572,  1.7371,  1.5784, -0.8376,  2.1668,  0.0709]],
       device='cuda:0')
layers.0.bias: tensor([-5.2871, -0.1532], device='cuda:0')
custom_layers.0.linear_sigmoid.weight: tensor([[-2.8622,  3.5002,  3.5002, -1.2077, -0.3514, -2.7014,  0.1911, -4.0444,
         -2.4567,  4.5526, -2.3954,  0.5576,  2.8622, -3.5002, -3.5002,  1.2077,
          0.3514,  2.7014, -0.1911,  4.0444,  2.4567, -4.5526,  2.3954, -0.5576],
        [-1.2468,  0.1080,  0.1080, -0.8848,  1.2655,  0.4896, -1.3350, -0.4135,
         -0.7122,  6.3164,  2.3583, -1.3577,  1

In [170]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
evaluate_model(model, custom_train_loader, criterion, optimizer)

Epoch 10, Loss: 0.034474270050600234
Validation Loss: 0.02292079052222107
Val Accuracy: 0.9920, Val F1-score: 0.9920

Epoch 20, Loss: 0.029318530854267666
Validation Loss: 0.020633031149448772
Val Accuracy: 0.9920, Val F1-score: 0.9920

Epoch 30, Loss: 0.027947709126820167
Validation Loss: 0.019263102932484344
Val Accuracy: 0.9934, Val F1-score: 0.9934

Epoch 40, Loss: 0.026803459035710103
Validation Loss: 0.0184571918958909
Val Accuracy: 0.9947, Val F1-score: 0.9947

Epoch 50, Loss: 0.0262053241051189
Validation Loss: 0.017474125321655265
Val Accuracy: 0.9947, Val F1-score: 0.9947

Epoch 60, Loss: 0.025492683169273184
Validation Loss: 0.01674341122856049
Val Accuracy: 0.9947, Val F1-score: 0.9947

Epoch 70, Loss: 0.02474595881091662
Validation Loss: 0.015327555445253438
Val Accuracy: 0.9947, Val F1-score: 0.9947

Epoch 80, Loss: 0.02411196921149928
Validation Loss: 0.01419157876420248
Val Accuracy: 0.9960, Val F1-score: 0.9960

Epoch 90, Loss: 0.023413314129443495
Validation Loss: 0.0

In [171]:
for name, param in model.named_parameters():
    print(f"{name}: {param.data}")

layers.0.weight: tensor([[-6.8731, -0.2201, -0.2201, -2.3613,  0.0142, -5.7033,  4.9509,  0.6240,
         -1.2830,  2.7564, -0.5424,  2.8271,  6.8731,  0.2201,  0.2201,  2.3613,
         -0.0142,  5.7033, -4.9509, -0.6240,  1.2830, -2.7564,  0.5424, -2.8271],
        [ 2.7096,  1.0830,  1.0830, -1.4308,  1.6737,  5.9750,  0.9735, -1.4588,
         -1.4514,  0.8855, -2.1203, -0.1945, -2.7096, -1.0830, -1.0830,  1.4308,
         -1.6737, -5.9750, -0.9735,  1.4588,  1.4514, -0.8855,  2.1203,  0.1945]],
       device='cuda:0')
layers.0.bias: tensor([-5.2766, -0.1409], device='cuda:0')
custom_layers.0.linear_sigmoid.weight: tensor([[-2.8046,  3.4831,  3.4831, -1.0879, -0.1621, -2.5287, -0.1675, -3.5384,
         -2.1596,  4.3878, -2.2100, -0.1946,  2.8046, -3.4831, -3.4831,  1.0879,
          0.1621,  2.5287,  0.1675,  3.5384,  2.1596, -4.3878,  2.2100,  0.1946],
        [ 0.1176,  0.1723,  0.1723, -0.3653,  0.4894,  0.4409, -0.6923,  0.0401,
         -0.1684,  2.3765,  0.9306, -0.6390, -0

In [162]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
evaluate_model(model, custom_train_loader, criterion, optimizer)

Epoch 10, Loss: 0.03519804935462566
Validation Loss: 0.02270101151104124
Val Accuracy: 0.9907, Val F1-score: 0.9907

Epoch 20, Loss: 0.029666416785951857
Validation Loss: 0.02001497138595217
Val Accuracy: 0.9920, Val F1-score: 0.9920

Epoch 30, Loss: 0.027499210976707725
Validation Loss: 0.01910037436393471
Val Accuracy: 0.9934, Val F1-score: 0.9934

Epoch 40, Loss: 0.026332393833173522
Validation Loss: 0.018331008238135382
Val Accuracy: 0.9947, Val F1-score: 0.9947

Epoch 50, Loss: 0.025513594991833897
Validation Loss: 0.017594824046928654
Val Accuracy: 0.9947, Val F1-score: 0.9947

Epoch 60, Loss: 0.024862841031362316
Validation Loss: 0.01695466785933301
Val Accuracy: 0.9947, Val F1-score: 0.9947

Epoch 70, Loss: 0.02431546530255382
Validation Loss: 0.016420449978037748
Val Accuracy: 0.9947, Val F1-score: 0.9947

Epoch 80, Loss: 0.02382908844492463
Validation Loss: 0.01600841553547146
Val Accuracy: 0.9947, Val F1-score: 0.9947

Epoch 90, Loss: 0.023368695846767405
Validation Loss: 0.

KeyboardInterrupt: 

In [156]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
evaluate_model(model, custom_train_loader, criterion, optimizer)

Epoch 10, Loss: 0.03440670182459702
Validation Loss: 0.022910369195869862
Val Accuracy: 0.9920, Val F1-score: 0.9920

Epoch 20, Loss: 0.030651397335147268
Validation Loss: 0.020970775986825174
Val Accuracy: 0.9907, Val F1-score: 0.9907

Epoch 30, Loss: 0.02788389798510127
Validation Loss: 0.019523509080765205
Val Accuracy: 0.9920, Val F1-score: 0.9920

Epoch 40, Loss: 0.026886521477237656
Validation Loss: 0.018615830441528185
Val Accuracy: 0.9934, Val F1-score: 0.9934

Epoch 50, Loss: 0.02604515223548434
Validation Loss: 0.01735817660843016
Val Accuracy: 0.9947, Val F1-score: 0.9947

Epoch 60, Loss: 0.025326186462544687
Validation Loss: 0.015755007010568534
Val Accuracy: 0.9960, Val F1-score: 0.9960

Epoch 70, Loss: 0.02461849710407502
Validation Loss: 0.014626990471801568
Val Accuracy: 0.9960, Val F1-score: 0.9960

Epoch 80, Loss: 0.024055235832701995
Validation Loss: 0.0138137694695691
Val Accuracy: 0.9960, Val F1-score: 0.9960

Epoch 90, Loss: 0.02321863984868032
Validation Loss: 0.

In [157]:
for name, param in model.named_parameters():
    print(f"{name}: {param.data}")

layers.0.weight: tensor([[-6.7005, -0.2762, -0.2762, -2.3480, -0.1286, -5.7162,  4.4786,  0.8280,
         -1.1978,  2.8410, -0.5636,  2.5976,  6.7005,  0.2762,  0.2762,  2.3480,
          0.1286,  5.7162, -4.4786, -0.8280,  1.1978, -2.8410,  0.5636, -2.5976],
        [ 2.9228,  1.0347,  1.0347, -1.4918,  1.6076,  5.9544,  0.4143, -1.2659,
         -1.3995,  1.2163, -1.9302, -0.4551, -2.9228, -1.0347, -1.0347,  1.4918,
         -1.6076, -5.9544, -0.4143,  1.2659,  1.3995, -1.2163,  1.9302,  0.4551]],
       device='cuda:0')
layers.0.bias: tensor([-5.5882, -0.3558], device='cuda:0')
custom_layers.0.linear_sigmoid.weight: tensor([[-0.7368,  3.4871,  3.4871, -1.1093, -0.4040, -1.1764,  0.2788, -3.3364,
         -2.0681,  3.1208, -5.5544,  0.5153,  0.7368, -3.4871, -3.4871,  1.1093,
          0.4040,  1.1764, -0.2788,  3.3364,  2.0681, -3.1208,  5.5544, -0.5153],
        [ 0.1622,  0.0336,  0.0336, -0.5137,  0.0892,  0.1958, -0.5015, -0.1368,
         -0.3423,  2.0886,  0.9950, -0.2834, -0

In [130]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
evaluate_model(model, custom_train_loader, criterion, optimizer)

Epoch 10, Loss: 0.04650792091464003
Validation Loss: 0.025719109962665243
Val Accuracy: 0.9947, Val F1-score: 0.9947

Epoch 20, Loss: 0.04112769169230926
Validation Loss: 0.021372774998781097
Val Accuracy: 0.9947, Val F1-score: 0.9947

Epoch 30, Loss: 0.038246722713531064
Validation Loss: 0.019369842237210833
Val Accuracy: 0.9947, Val F1-score: 0.9947

Epoch 40, Loss: 0.036295082384346866
Validation Loss: 0.018089132215663994
Val Accuracy: 0.9947, Val F1-score: 0.9947

Epoch 50, Loss: 0.03483528974232733
Validation Loss: 0.01716014178517374
Val Accuracy: 0.9947, Val F1-score: 0.9947

Epoch 60, Loss: 0.033692508762383284
Validation Loss: 0.016447162977480428
Val Accuracy: 0.9947, Val F1-score: 0.9947

Epoch 70, Loss: 0.03277765942127511
Validation Loss: 0.015881943373263614
Val Accuracy: 0.9947, Val F1-score: 0.9947

Epoch 80, Loss: 0.032035941505397666
Validation Loss: 0.01542384399507076
Val Accuracy: 0.9960, Val F1-score: 0.9960

Epoch 90, Loss: 0.031429176907848584
Validation Loss: 

In [131]:
for name, param in model.named_parameters():
    print(f"{name}: {param.data}")

layers.0.weight: tensor([[-6.5369,  0.5586,  0.5586,  7.3472,  0.0885, -6.7856, -0.0075, -5.7633,
          1.5288,  0.7641,  1.4206,  0.2905,  6.5369, -0.5586, -0.5586, -7.3472,
         -0.0885,  6.7856,  0.0075,  5.7633, -1.5288, -0.7641, -1.4206, -0.2905],
        [ 6.5367, -0.5586, -0.5586, -7.3478, -0.0883,  6.7859,  0.0076,  5.7627,
         -1.5294, -0.7648, -1.4203, -0.2906, -6.5367,  0.5586,  0.5586,  7.3478,
          0.0883, -6.7859, -0.0076, -5.7627,  1.5294,  0.7648,  1.4203,  0.2906]],
       device='cuda:0')
layers.0.bias: tensor([-3.5744,  3.5738], device='cuda:0')
